In [1]:
REPO_NAME="big-data-lab"
USER_NAME="SyPa97" # TODO: replace with your GitHub username

In [2]:
%env REPO_NAME=$REPO_NAME
%env USER_NAME=$USER_NAME

env: REPO_NAME=big-data-lab
env: USER_NAME=SyPa97


In [3]:
%%shell
rm -rf $REPO_NAME
git clone https://github.com/$USER_NAME/$REPO_NAME.git
cd $REPO_NAME

Cloning into 'big-data-lab'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 111 (delta 28), reused 13 (delta 13), pack-reused 68 (from 2)
Receiving objects: 100% (111/111), 448.47 KiB | 1.76 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [4]:
%run $REPO_NAME/notebooks/lib/00_terraform_setup.ipynb

env: TERRAFORM_VERSION=1.9.8
env: PATH=/content/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
--2024-12-21 13:31:19--  https://releases.hashicorp.com/terraform/1.9.8/terraform_1.9.8_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 54.230.21.53, 54.230.21.14, 54.230.21.37, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|54.230.21.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27041434 (26M) [application/zip]
Saving to: ‘terraform_1.9.8_linux_amd64.zip’

terraform_1.9.8_lin 100%[===================>]  25.79M  60.2MB/s    in 0.4s    

2024-12-21 13:31:20 (60.2 MB/s) - ‘terraform_1.9.8_linux_amd64.zip’ saved [27041434/27041434]

Archive:  terraform_1.9.8_linux_amd64.zip
  inflating: bin/LICENSE.txt         
  inflating: bin/terraform           
Terraform v1.9.8
on linux_amd64

Your version of Terraform 

In [5]:
%run $REPO_NAME/notebooks/conf/00_env_variables.ipynb

env: TF_VAR_ds_semester=2024l
env: TF_VAR_user_id=7033
env: TF_VAR_billing_account=01111F-DBFD00-411C42
env: TF_VAR_region=europe-west1
env: TF_VAR_project_name=ds-2024l-7033
env: GIT_BRANCH=colab-dev


In [6]:
# Authenticate to the Google account
from google.colab import auth
auth.authenticate_user()

### Set the remote bucket for storing Terraform state


In [7]:
import os
TF_VAR_user_id = os.getenv("TF_VAR_user_id")
TF_VAR_user_id

'7033'

In [8]:
import subprocess

command = f"cd {REPO_NAME}/infra && terraform init -backend-config=bucket=ds-2024l-{TF_VAR_user_id}-state"

result = subprocess.run(command, shell=True, check=True)
print("Komenda zakończona sukcesem", result)

Komenda zakończona sukcesem CompletedProcess(args='cd big-data-lab/infra && terraform init -backend-config=bucket=ds-2024l-7033-state', returncode=0)


### Provision the infrastructure

In [9]:
%%shell
cd $REPO_NAME/infra
terraform apply -auto-approve

Acquiring state lock. This may take a few moments...

Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # module.dataproc.google_dataproc_cluster.dataproc-cluster will be created
  + resource "google_dataproc_cluster" "dataproc-cluster" {
      + effective_labels              = (known after apply)
      + graceful_decommission_timeout = "0s"
      + id                            = (known after apply)
      + name                          = "ds-cluster"
      + project                       = "ds-2024l-7033"
      + region                        = "europe-west1"
      + terraform_labels              = (known after apply)

      + cluster_config {
          + bucket      = (known after apply)
          + temp_bucket = (known after apply)

          + auxiliary_node_groups (known after apply)

          + dataproc_metric_config (known after 

### Destroy the infrastructure
After you are done with the lab, you can destroy the infrastructure by running the cell below:

In [10]:
%%shell
cd $REPO_NAME/infra
terraform destroy -auto-approve

Acquiring state lock. This may take a few moments...
module.vpc.module.vpc.module.vpc.google_compute_network.network: Refreshing state... [id=projects/ds-2024l-7033/global/networks/main-vpc]
module.vpc.google_compute_firewall.fw-allow-ingress-from-iap: Refreshing state... [id=projects/ds-2024l-7033/global/firewalls/fw-allow-ingress-iap]
module.vpc.module.vpc.module.subnets.google_compute_subnetwork.subnetwork["europe-west1/dataproc-01"]: Refreshing state... [id=projects/ds-2024l-7033/regions/europe-west1/subnetworks/dataproc-01]
module.vpc.google_compute_firewall.default-internal-allow-all: Refreshing state... [id=projects/ds-2024l-7033/global/firewalls/default-internal-allow-all]
module.vpc.module.cloud-router.google_compute_router.router: Refreshing state... [id=projects/ds-2024l-7033/regions/europe-west1/routers/nat-router]
module.vpc.module.vpc.module.routes.google_compute_route.route["egress-internet"]: Refreshing state... [id=projects/ds-2024l-7033/global/routes/egress-internet]
